In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse

In [2]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--conv-in", type=int, default=4, help="Input sequence features"
    )
    parser.add_argument("--batch_size", type=int, default=512, required=False)
    parser.add_argument("--step_interval", type=int, default=100, required=False)
    parser.add_argument("--lr", type=float, default=0.001, required=False)
    parser.add_argument(
        "--train-file",
        type=str,
        help="feature file used in trainning",
        default="/home/xiaoyf/methylation/deepsignal/log/feature.txt",
    )
    parser.add_argument(
        "--model_dir", type=str, default="/home/xiaoyf/methylation/deepsignal/log/"
    )
    parser.add_argument(
        "--max_epoch_num",
        action="store",
        default=10,
        type=int,
        required=False,
        help="max epoch num, default 10",
    )
    parser.add_argument(
        "--min_epoch_num",
        action="store",
        default=5,
        type=int,
        required=False,
        help="min epoch num, default 5",
    )
    return parser.parse_args([])

In [3]:
def squash(tensor, dim=-1):
    squared_norm = (tensor**2).sum(dim=dim, keepdim=True)
    scale = squared_norm / (1 + squared_norm)
    return scale * tensor / torch.sqrt(squared_norm)

In [4]:
class Squash(nn.Module):
    def __init__(self, eps=10e-21, **kwargs):
        super(Squash, self).__init__(**kwargs)
        self.eps = eps

    def forward(self, s):
        n = torch.norm(s, dim=-1, keepdim=True)
        return (1 - 1 / (torch.exp(n) + self.eps)) * (s / (n + self.eps))

In [112]:
def dynamic_routing(x, iterations=3):
    #x = x.unsqueeze(-1)
    N = x.shape[1]  # num_caps
    N1 = x.shape[2]  # in_caps
    B = x.shape[0]
    # feature_dim = x.shape[2]
    #x:batch_size, num_caps, in_caps, out_channels
    b = torch.zeros(B, N, N1,1).to(x.device)#batch_size, num_caps, in_caps
    for _ in range(iterations):
        #print('input x\'s batch_size: {}, num_caps: {}, in_caps: {}, out_channels: {}'.format(x.shape[0], x.shape[1], x.shape[2], x.shape[3]))
        c = F.softmax(b, dim=1)#Softmax along num_caps
        #batch_size, num_caps,caps_dim
        #print('softmax result\'s batch_size: {}, num_caps: {}, in_caps: {}, softmax_result: {}'.format(c.shape[0], c.shape[1], c.shape[2], c.shape[3]))
        a = c*x
        #print('a\'s batch_size: {}, num_caps: {}, in_caps: {}, out_channels: {}'.format(a.shape[0], a.shape[1], a.shape[2], a.shape[3]))
        s = torch.sum(a, dim=2).squeeze(-1)#sum across in_caps
        #print('s\'s batch_size: {}, num_caps: {}, out_channels: {}'.format(s.shape[0], s.shape[1], s.shape[2]))
        v = squash(s)# apply "squashing" non-linearity along out_channels
        #print('v\'s batch_size: {}, num_caps: {}, out_channels: {}'.format(v.shape[0], v.shape[1], v.shape[2]))
        #print('x shape: {}'.format(x.shape))
        y = torch.matmul(x,v.unsqueeze(-1))
        #print('y shape: {}'.format(y.shape))
        #print('b shape: {}'.format(b.shape))
        b = b + y

    return v

In [110]:
#x:batch_size, num_caps, in_caps, out_channels
input_tensor=torch.randn(2, 2, 5,205)
print(dynamic_routing(input_tensor).shape)

input x's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
softmax result's batch_size: 2, num_caps: 2, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
s's batch_size: 2, num_caps: 2, out_channels: 205
v's batch_size: 2, num_caps: 2, out_channels: 205
x shape: torch.Size([2, 2, 5, 205])
y shape: torch.Size([2, 2, 5, 1])
b shape: torch.Size([2, 2, 5, 1])
input x's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
softmax result's batch_size: 2, num_caps: 2, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
s's batch_size: 2, num_caps: 2, out_channels: 205
v's batch_size: 2, num_caps: 2, out_channels: 205
x shape: torch.Size([2, 2, 5, 205])
y shape: torch.Size([2, 2, 5, 1])
b shape: torch.Size([2, 2, 5, 1])
input x's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
softmax result's batch_size: 2, num_caps: 2, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 2, in_caps

In [ ]:
import torch.nn as nn

# Define the input and output channels
in_channels = 2
out_channels = 1

# Define the kernel size and dilation
kernel_size = 2

# Define the 1D dilated convolution layers
conv1d_list = nn.ModuleList()
for dilation in range(1, 6):
    padding = "same"
    conv1d_list.append(
        nn.Conv1d(
            in_channels, out_channels, kernel_size, dilation=dilation, padding=padding
        )
    )

# Define the input tensor
input_tensor = torch.randn(1, in_channels, 21 * 5)

# Apply the 1D dilated convolutions to the input tensor
output_tensor_list = []
for conv1d in conv1d_list:
    print(conv1d(input_tensor).shape)
    output_tensor_list.append(conv1d(input_tensor))

# Concatenate the output tensors along the channel dimension
output_tensor = torch.cat(output_tensor_list, dim=1)

print(output_tensor.shape)
print(dynamic_routing(output_tensor).shape)

In [16]:
c = F.softmax(torch.zeros(1, 1, 5, 1, 1), dim=1)
# print(c)
print(c.shape)
x = torch.randn(1, 5, 21 * 5, 1)
a = x.matmul(c)
# print(a)
print(a.shape)
s = torch.sum(x.matmul(c), dim=2).squeeze(-1)
# print(s)
print(s.shape)

torch.Size([1, 1, 5, 1, 1])
torch.Size([1, 1, 5, 105, 1])
torch.Size([1, 1, 105])


In [127]:
class PrimaryCapsuleLayer(nn.Module):
    """
    Create a primary capsule layer with the methodology described in 'Efficient-CapsNet: Capsule Network with Self-Attention Routing'.
    Properties of each capsule s_n are exatracted using a 1D depthwise convolution.

    ...

    Attributes
    ----------
    kernel_size[w]: int
        depthwise conv kernel dimension
    conv_num: int
        number of primary capsules
    feature_dimension: int
        primary capsules dimension (number of properties)
    conv_stride: int
        depthwise conv strides
    Methods
    -------
    forward(inputs)
        compute the primary capsule layer
    """

    def __init__(
        self,
        conv_in=2,
        feature_dimension=21 * 5,
        kernel_size=2,
        conv_num=5,
        base_num=21,
    ):
        super().__init__()
        
        self.conv_out = feature_dimension // (conv_num * base_num)
        self.conv_num = conv_num
        self.primary_capsule_layer = nn.ModuleList(
            [
                nn.Conv1d(
                    conv_in,
                    self.conv_out,
                    kernel_size,
                    dilation=conv_stride,
                    padding="same",
                )
                for conv_stride in range(1, conv_num + 1)
            ]
        )

    def forward(self, x):
        
        #print('input feature shape: {}'.format(x.shape))
        capsules = [conv(x) for conv in self.primary_capsule_layer]
        # capsules_reshaped = [
        #    c.reshape(self.conv_num, self.feature_dimension) for c in capsules
        # ]
        output_tensor = torch.cat(capsules, dim=1)
        return Squash()(output_tensor)


def test_for_primary_capsule_layer():
    input = torch.rand(1, 2, 105)
    layer = PrimaryCapsuleLayer()
    print(layer(input).shape)


test_for_primary_capsule_layer()

torch.Size([1, 5, 105])


In [113]:
class CapsLayer(nn.Module):
    def __init__(
        self, num_capsules=1, in_caps=5, in_channels=105, out_channels=2
    ):
        super(CapsLayer, self).__init__()
        self.W = nn.Parameter(
            0.01
            * torch.randn(1, num_capsules, in_caps, out_channels, in_channels)
        )
        # print('W shape: {}'.format(self.W.shape))

    def forward(self, x):
        #print('CapsLayer input shape: {}'.format(x.shape))
        x = x[:, None, ..., None]  # x.unsqueeze(1).unsqueeze(4)
        # x = x.unsqueeze(-1)
        #print('W shape: {}'.format(self.W.shape))
        #print('CapsLayer input shape: {}'.format(x.shape))
        # print('CapsLayer input expand shape: {}'.format(x[ :, :, None, :].shape))
        # (batch_size, num_caps, num_route_nodes, out_channels, 1)
        # print('x shape: {}'.format(x.shape))
        u_hat = torch.matmul(self.W, x)  # (x @ self.W).squeeze(2)
        # u=u_hat.squeeze(-1)
        u_hat = u_hat.squeeze(-1)
        #batch_size, num_caps, in_caps, out_channels
        #print('u_hat\'s batch_size: {}, num_caps: {}, in_caps: {}, out_channels: {}'.format(u_hat.shape[0], u_hat.shape[1], u_hat.shape[2], u_hat.shape[3]))
        class_capsules = dynamic_routing(u_hat)
        return class_capsules

In [54]:
a = torch.rand(1, 10, 10, 20, 1)
b = torch.rand(1, 1, 10, 1, 1)
c = a.matmul(b)

In [107]:
input = torch.rand(2, 5, 105)
layer = CapsLayer()
print(layer(input).shape)

CapsLayer input shape: torch.Size([2, 5, 105])
W shape: torch.Size([1, 1, 5, 2, 105])
CapsLayer input shape: torch.Size([2, 1, 5, 105, 1])
u_hat's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
input x's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
softmax result's batch_size: 2, num_caps: 1, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
s's batch_size: 2, num_caps: 1, out_channels: 2
v's batch_size: 2, num_caps: 1, out_channels: 2
x shape: torch.Size([2, 1, 5, 2])
y shape: torch.Size([2, 1, 5, 1])
b shape: torch.Size([2, 1, 5, 1])
input x's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
softmax result's batch_size: 2, num_caps: 1, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
s's batch_size: 2, num_caps: 1, out_channels: 2
v's batch_size: 2, num_caps: 1, out_channels: 2
x shape: torch.Size([2, 1, 5, 2])
y shape: torch.Size([2, 1, 5, 1])
b shape: torch.Size([2, 1, 5, 1])

In [139]:
class CapsNet(nn.Module):
    def __init__(self,vocab_size=4,embedding_size=1):
        super(CapsNet, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.primary_layer = PrimaryCapsuleLayer()
        self.caps_layer = CapsLayer()
        self.softmax = nn.Softmax(1)

    def forward(self, seq,sig):
        seq_emb = self.embed(seq.long())
        seq_emb = seq_emb.reshape(seq_emb.shape[0], 1, -1)
        sig = sig.reshape(sig.shape[0], 1, -1)
        #print('seq_emb shape: {}'.format(seq_emb.shape))
        #print('sig shape: {}'.format(sig.shape))
        x = torch.cat((seq_emb,sig), dim=1)
        #seq = self.primary_layer(seq)
        #seq = self.caps_layer(seq)
        #sig = self.primary_layer(sig)
        #sig = self.caps_layer(sig)
        x = self.primary_layer(x)
        x = self.caps_layer(x)
        x = torch.norm(x,dim=-1)
        return x


def test_for_caps_net():
    input1 = torch.rand(1, 1, 105)
    input2 = torch.rand(1, 1, 105)
    
    model = CapsNet()
    #print(model(input1,input2).shape)


test_for_caps_net()

In [141]:
class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()

    def forward(self, classes, labels):
        #classes = classes.reshape(classes.shape[0], 2)
        labels = labels.reshape(labels.shape[0], 1)
        #print('classes shape: {}'.format(classes.shape))
        #print('labels shape: {}'.format(labels.shape))
        left = F.relu(0.9 - classes, inplace=True) ** 2
        #print('left shape: {}'.format(left.shape))
        right = F.relu(classes - 0.1, inplace=True) ** 2
        #print('right shape: {}'.format(right.shape))

        margin_loss = labels * left + 0.5 * (1.0 - labels) * right
        margin_loss = margin_loss.sum()
        return margin_loss

In [17]:
np.random.randint(0, 2)

1

In [49]:
from torch.utils.data import Dataset
import linecache

base2code_dna = {"A": 0, "C": 1, "G": 2, "T": 3, "N": 4}
code2base_dna = {0: "A", 1: "C", 2: "G", 3: "T", 4: "N"}


def clear_linecache():
    # linecache should be treated carefully
    linecache.clearcache()


def parse_a_line2(line):
    words = line.strip().split("\t")

    seq = np.array(
        [[base2code_dna[y] for y in x.split(",")] for x in words[1].split(";")]
    )
    signal = np.array(
        [[np.float16(y) for y in x.split(",")] for x in words[2].split(";")]
    )
    label = np.random.randint(0, 2)

    return seq, signal, label


class SignalDataset(Dataset):
    def __init__(self, filename, transform=None):
        # print(">>>using linecache to access '{}'<<<\n"
        #       ">>>after done using the file, "
        #       "remember to use linecache.clearcache() to clear cache for safety<<<".format(filename))
        self._filename = os.path.abspath(filename)
        self._total_data = 0
        self._transform = transform
        with open(filename, "r") as f:
            self._total_data = len(f.readlines())

    def __getitem__(self, idx):
        line = linecache.getline(self._filename, idx + 1)
        if line == "":
            return None
        else:
            output = parse_a_line2(line)
            if self._transform is not None:
                output = self._transform(output)
            return output

    def __len__(self):
        return self._total_data

In [144]:
import sys
from torch.optim.lr_scheduler import StepLR
import time
from sklearn import metrics
import re

if __name__ == "__main__":
    total_start = time.time()
    args = parse_args()
    train_dataset = SignalDataset(args.train_file)
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=args.batch_size, shuffle=False
    )
    total_step = len(train_loader)
    valid_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=args.batch_size, shuffle=True
    )
    model = CapsNet()
    criterion = CapsuleLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    scheduler = StepLR(optimizer, step_size=2, gamma=0.1)
    curr_best_accuracy = 0
    model_dir = args.model_dir
    if model_dir != "/":
        model_dir = os.path.abspath(model_dir).rstrip("/")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        else:
            model_regex = re.compile(
                r"" + "\.b\d+_s\d+_epoch\d+\.ckpt*"
            )
            for mfile in os.listdir(model_dir):
                if model_regex.match(mfile):
                    os.remove(model_dir + "/" + mfile)
        model_dir += "/"
    model.train()
    for epoch in range(args.max_epoch_num):
        curr_best_accuracy_epoch = 0
        no_best_model = True
        tlosses = []
        start = time.time()
        for i, sfeatures in enumerate(train_loader):
            (seq, signal, labels) = sfeatures
            outputs = model(seq, signal)
            loss = criterion(outputs, labels)
            tlosses.append(loss.detach().item())

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            if (i + 1) % args.step_interval == 0 or i == total_step - 1:
                model.eval()
                with torch.no_grad():
                    vlosses, vlabels_total, vpredicted_total = [], [], []
                    for vi, vsfeatures in enumerate(valid_loader):
                        (
                            vseq,
                            vsignal,
                            vlabels,
                        ) = vsfeatures
                        voutputs = model(vseq, vsignal)
                        vloss = criterion(voutputs, vlabels)

                        _, vpredicted = torch.max(voutputs.data, 1)
                        #print(vpredicted)
                        vlosses.append(vloss.item())
                        vlabels_total += vlabels.tolist()
                        vpredicted_total += vpredicted.tolist()
                        v_accuracy = metrics.accuracy_score(
                            vlabels_total, vpredicted_total
                        )
                        v_precision = metrics.precision_score(
                            vlabels_total, vpredicted_total
                        )
                        v_recall = metrics.recall_score(vlabels_total, vpredicted_total)
                        if v_accuracy > curr_best_accuracy_epoch:
                            curr_best_accuracy_epoch = v_accuracy
                            if curr_best_accuracy_epoch > curr_best_accuracy - 0.0002:
                                torch.save(
                                    model.state_dict(),
                                    model_dir
                                    + ".epoch{}.ckpt".format(
                                        epoch
                                    ),
                                )
                                if curr_best_accuracy_epoch > curr_best_accuracy:
                                    curr_best_accuracy = curr_best_accuracy_epoch
                                    no_best_model = False
                        time_cost = time.time() - start
                        print(
                            "Epoch [{}/{}], Step [{}/{}], TrainLoss: {:.4f}; "
                            "ValidLoss: {:.4f}, "
                            "Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, "
                            "curr_epoch_best_accuracy: {:.4f}; Time: {:.2f}s".format(
                                epoch + 1,
                                args.max_epoch_num,
                                i + 1,
                                total_step,
                                np.mean(tlosses),
                                np.mean(vlosses),
                                v_accuracy,
                                v_precision,
                                v_recall,
                                curr_best_accuracy_epoch,
                                time_cost,
                            )
                        )
                        tlosses = []
                        start = time.time()
                        sys.stdout.flush()
                    model.train()
            scheduler.step()
            if no_best_model and epoch >= args.min_epoch_num - 1:
                print("early stop!")
                break
        endtime = time.time()
        clear_linecache()
        print(
            "[main] train costs {} seconds, "
            "best accuracy: {}".format(endtime - total_start, curr_best_accuracy)
        )

Epoch [1/10], Step [100/1421], TrainLoss: 196.5193; ValidLoss: 206.4992, Accuracy: 0.4727, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.4727; Time: 14.27s


/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/10], Step [100/1421], TrainLoss: nan; ValidLoss: 200.0250, Accuracy: 0.4893, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.4893; Time: 0.28s
Epoch [1/10], Step [100/1421], TrainLoss: nan; ValidLoss: 199.1698, Accuracy: 0.4915, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.4915; Time: 0.09s


/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

Epoch [1/10], Step [100/1421], TrainLoss: nan; ValidLoss: 197.8267, Accuracy: 0.4951, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.4951; Time: 0.08s
Epoch [1/10], Step [100/1421], TrainLoss: nan; ValidLoss: 198.7946, Accuracy: 0.4926, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.4951; Time: 0.11s


...

Epoch [1/10], Step [100/1421], TrainLoss: nan; ValidLoss: 195.9087, Accuracy: 0.5000, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.5053; Time: 1.88s


/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/10], Step [100/1421], TrainLoss: nan; ValidLoss: 195.9172, Accuracy: 0.4999, Precision: 0.0000, Recall: 0.0000, curr_epoch_best_accuracy: 0.5053; Time: 1.82s


/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/xiaoyf/anaconda3/envs/deepsignal/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 